In [1]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.palettes import Paired, magma, Plasma256, viridis, linear_palette, Category20
import squarify
import json
from ete3 import Tree 

In [2]:
#import file to t
t = Tree("NewickTree.nwk")

In [9]:
print(t)


      /-raccoon
   /-|
  |   \-bear
  |
  |      /-sea_lion
  |   /-|
  |  |   \-seal
--|--|
  |  |      /-monkey
  |  |   /-|
  |   \-|   \-cat
  |     |
  |      \-weasel
  |
   \-dog


In [8]:
a=0
BuildTree(t)
show(p)

In [3]:
output_file("square.html")
p = figure(plot_width=1600, plot_height=1000)
Border = []
Colors = viridis(15)
def BuildTree(node):
    global a 
    if node.is_root():
        TempBorder = [0,0,500,700]
        ParentCoordinates = GetCoor([len(node)], TempBorder)
        RootColor = Colors[a]
        a = a+1
        Border.append(ParentCoordinates[0])
        Drawing(ParentCoordinates[0], RootColor)
        TraverseNode(node)
            
    elif node.is_leaf() == False  & node.is_root() == False:
        TraverseNode(node)

In [4]:
def TraverseNode(node):
    global a
    weights = []
    NodeBorder = Border[-1]
    NodeBorder[0] = NodeBorder[0]+10  
    NodeBorder[1] = NodeBorder[1]+10  
    NodeBorder[2] = NodeBorder[2]-10  
    NodeBorder[3] = NodeBorder[3]-10  
    
    for i in range(0,len(node.get_children())):
        weights.append(len(node.children[i]))
    ChildCoordinates = GetCoor(weights, NodeBorder)
    for i in range(0,len(node.get_children())):       
        ChildColor = Colors[a]
        a = a+1
        Drawing(ChildCoordinates[i], ChildColor)
        Border.append(ChildCoordinates[i])
        BuildTree(node.children[i])
        del Border[-1]

In [5]:
def GetCoor(vals, MyBorder): 
    # these values define the coordinate system for the returned rectangles
    # the values will range from x to x + width and y to y + height
    border_x = MyBorder[0]
    border_y = MyBorder[1]
    border_width = MyBorder[2]-MyBorder[0]
    border_height = MyBorder[3]-MyBorder[1]

    values = vals

    # values must be sorted descending (and positive, obviously)
    values.sort(reverse=True)

    # the sum of the values must equal the total area to be laid out
    values = squarify.normalize_sizes(values, border_width, border_height)
    rects = squarify.squarify(values, border_x, border_y, border_width, border_height)
    
    #values assigned to coordinates
    rect = []
    for i in range(len(rects)):
        temp = []
        temp.append(rects[i].get("x"))
        temp.append(rects[i].get("y"))
        temp.append(rects[i].get("dx")+rects[i].get("x"))
        temp.append(rects[i].get("dy")+rects[i].get("y"))
        rect.append(temp)
    return rect

In [6]:
def Drawing(Coordinates, color):
    p.quad(top = Coordinates[3], 
       bottom = Coordinates[1] , 
       left = Coordinates[0], 
       right = Coordinates[2], 
       color = color,
           line_color = 'white',
           line_width = 3,)  
    p.grid.visible = False
    p.axis.visible = False

In [7]:
def SetUpColor(node):
    counter = 0
    for i in node.traverse("postorder"):
        counter += 1
    return counter